In [1]:
from dlinputs import tarrecords
import imp
imp.reload(tarrecords)
import numpy as np

In [2]:
stream = open("testdata/imagenet-000000.tgz", "rb")
data = tarrecords.tardata(stream)
samples = list(data)
assert samples[0] == ('10.cls', '304'.encode("ascii")), samples[0]
assert {2} == set([len(x) for x in samples])

In [3]:
stream = open("testdata/imagenet-000000.tgz", "rb")
data = tarrecords.tardata(stream)
data = tarrecords.group_by_keys()(data)
samples = list(data)
print(list(samples[0].keys()))

['__key__', 'cls', 'png', 'wnid', 'xml']


In [4]:
!tar -ztvf testdata/imagenet-000000.tgz | sed 7q
!tar xvf testdata/imagenet-000000.tgz 10.png
!file 10.png

-rw-r--r-- tmb/tmb           3 1969-12-31 16:00 10.cls
-rw-rw-r-- tmb/tmb       75442 2018-04-16 10:21 10.png
-rw-r--r-- tmb/tmb           9 1969-12-31 16:00 10.wnid
-rw-r--r-- tmb/tmb           4 1969-12-31 16:00 10.xml
-rw-r--r-- tmb/tmb           3 1969-12-31 16:00 12.cls
-rw-rw-r-- tmb/tmb       80108 2018-04-16 10:21 12.png
-rw-r--r-- tmb/tmb           9 1969-12-31 16:00 12.wnid
tar: write error
10.png
10.png: PNG image data, 600 x 793, 8-bit/color RGB, non-interlaced


In [5]:
stream = open("testdata/imagenet-000000.tgz", "rb")
data = tarrecords.tardata(stream)
data = tarrecords.group_by_keys()(data)
data = tarrecords.decoder()(data)
samples = list(data)
print(list(samples[0].keys()))

['__key__', 'cls', 'png', 'wnid', 'xml']


In [6]:
stream = open("testdata/imagenet-000000.tgz", "rb")
data = tarrecords.tariterator1(stream)
samples = list(data)
assert len(samples)==47
assert samples[0]["__key__"] == "10", samples[0]["__key__"]
assert set(samples[3].keys()) == set("__key__ png cls xml wnid".split()), list(samples[3].keys())
assert samples[-1]["png"].shape == (400, 300, 3)

In [7]:
stream = open("testdata/imagenet-000000.tgz", "rb")
data = tarrecords.tariterator(stream)
samples = list(data)
assert len(samples)==47
for i in range(len(samples)):
    assert samples[i]["png"].dtype == np.dtype('f'), samples[i]["png"].dtype
    assert np.amin(samples[i]["png"]) >= 0, np.amin(samples[i]["png"])
    assert np.amin(samples[i]["png"]) <= 1, np.amax(samples[i]["png"])
assert samples[0]["__key__"] == "10"
assert set(samples[3].keys()) == set("__key__ __source__ cls png xml wnid".split()), list(samples[3].keys())
assert samples[-1]["png"].shape == (400, 300, 3)

In [8]:
stream = open("/tmp/test.tgz", "wb")
sink = tarrecords.TarWriter(stream)
for sample in samples:
    sink.write(sample)
sink.close()
stream.close()
!ls -l /tmp/test.tgz

-rw-r--r-- 1 tmb tmb 12993065 Mar 26 11:31 /tmp/test.tgz


In [9]:
stream = open("/tmp/test.tgz", "rb")
data = tarrecords.tariterator(stream)
samples = list(data)
assert len(samples)==47
assert samples[0]["__key__"] == "10"
assert set(samples[3].keys()) == set("__key__ __source__ cls png xml wnid".split()), list(samples[3].keys())
assert samples[-1]["png"].shape == (400, 300, 3)